In [3]:
import pandas as pd
import geopandas as gpd


import requests


import os



from geojson import FeatureCollection
import shapely

##### IMPORT LOCAL DATA


In [3]:
bars = pd.read_excel("nashville-bars-data.xlsx")
MAPBOX_TOKEN = os.getenv("MAPBOX_TOKEN")

In [4]:
def mapboxGeocode(params: dict = {}):
    baseUrl = "https://api.mapbox.com/search/geocode/v6/forward"
    req = requests.get(baseUrl, params=params)
    return req.json()


def getGeocodeCoordinates(result: FeatureCollection):
    return ",".join(([str(x) for x in result['features'][0]["geometry"]["coordinates"]]))


def getAddressCoordinates(address):
    mapbox_query = {
        "q": address,
        "access_token": MAPBOX_TOKEN,
        "country": 'US'
    }
    geocode_results = mapboxGeocode(mapbox_query)
    string_coordinates = getGeocodeCoordinates(geocode_results)
    return string_coordinates

In [5]:
bars['coordinates'] = bars["address"].apply(lambda x: getAddressCoordinates(x))
bars['longitude'] = bars["coordinates"].apply(lambda x: float(x.split(",")[0]))
bars['latitude'] = bars["coordinates"].apply(lambda x: float(x.split(",")[1]))

In [6]:
bars.to_json("nashville-bars-data.json", orient="records")

In [2]:
bars = pd.read_json("nashville-bars-data.json")

In [ ]:
bars['geometry'] = bars.apply(lambda x:  shapely.Point(
    [x['longitude'], x['latitude']]), axis=1)

In [7]:
bars_gdf = gpd.GeoDataFrame(bars, geometry="geometry")

In [8]:
zipcodes_gdf = gpd.read_file("nashville_zipcode_refined.json")

In [26]:
def getProFromIntersect(k: gpd.GeoSeries, t: gpd.GeoDataFrame, p: str):
    loc: list[shapely.Polygon | shapely.MultiPolygon] = list(t.geometry)
    count = 0
    found = False
    while not found:
        item_loc = loc[count]
        found = item_loc.intersects(k.geometry)
        if (found):
            break
        else:
            count += 1

    return t[p][count]

In [ ]:
bars_gdf['zipcode'] = bars_gdf.apply(
    lambda x: getProFromIntersect(x, zipcodes_gdf, "zipcode"), axis=1)

In [29]:
bars_gdf.to_file("nashville-bars-data.geojson", driver="GeoJSON")